In [13]:
# 创建clent
from openai import OpenAI
from dotenv import load_dotenv

import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key,base_url="https://api.deepseek.com")

In [14]:
messages=[{"role": "user", "content": "Explain the importance of fast language models"}]

chat_completion = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    temperature=0,
)

chat_completion.choices[0].message.content

'Fast language models are crucial for several reasons, impacting both user experience and practical applications:\n\n### 1. **Real-Time Interaction**  \n   - **Conversational AI (Chatbots, Assistants):** Users expect instant responses in applications like customer support or virtual assistants (e.g., ChatGPT, Alexa). Delays disrupt the flow of conversation and reduce usability.  \n   - **Gaming/NPCs:** Fast responses are needed for immersive interactions with AI-driven characters.  \n\n### 2. **Scalability & Cost Efficiency**  \n   - **Lower Compute Costs:** Faster inference reduces the computational resources (and energy) required per query, making deployment cheaper at scale (e.g., for cloud services).  \n   - **Higher Throughput:** Speed allows serving more users simultaneously without latency spikes, critical for apps with millions of users.  \n\n### 3. **Enhanced Productivity**  \n   - **Coding Tools (Copilot, CodeLlama):** Developers rely on near-instant code suggestions to maint

In [15]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        compiletion = self.client.chat.completions.create(
            model="deepseek-chat",
            messages=self.messages,
        )
        return compiletion.choices[0].message.content

In [16]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [17]:
def calculate(operation) -> float:
    return eval(operation)

def get_planet_mass(planet) -> float:
    if planet == "earth":
        return 5.972e24
    if planet == "mars":
        return 6.39e23
    if planet == "jupiter":
        return 1.898e27
    if planet == "saturn":
        return 5.683e26
    if planet == "uranus":
        return 8.681e25
    if planet == "neptune":
        return 1.024e26
    if planet == "mercury":
        return 3.285e23
    if planet == "venus":
        return 4.867e24
    return None

In [18]:
agent_demo = Agent(client=client, system=system_prompt)

In [19]:
result = agent_demo("what is the mass of Mercury times 5")
print(result)

Thought: I need to find the mass of Mercury first.
Action: get_planet_mass: Mercury
PAUSE


In [22]:
result = get_planet_mass("mercury")
print(result)

3.285e+23


In [23]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: 3.285e+23'

In [24]:
result = agent_demo(next_prompt)
print(result)

Thought: Now I need to multiply the mass of Mercury by 5.
Action: calculate: 3.285e+23 * 5
PAUSE


In [25]:
result = calculate("3.285e+23 * 5")
print(result)

1.6425e+24


In [26]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: 1.6425e+24'

In [28]:
result = agent_demo(next_prompt)
result

'Answer: The mass of Mercury times 5 is 1.6425e+24 kg.'

In [30]:
for msg in agent_demo.messages:
    print(msg['content'])

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 